In [1]:
# 1. SQL에 임베딩해놓은 파일 임포트 2. DB에서 임베딩 컬럼 조회 후 가져오고 3. 코사인 유사도 비교

import pandas as pd
import numpy as np

### 데이터 불러오기

> train data

In [2]:
df = pd.read_json('train_emd.json')
df.head(10)

,tags,id,plylst_title,songs,like_cnt,updt_date,songs_cnt,embedding
0,[팝],50564,1 HITS OF DECADE SPECIAL 2,"[60025, 326802, 82991, 533287, 132109, 298235,...",5,2009-07-11 21:37:42.000,25,"[-0.335850656, -0.4305538535, 0.44612923260000..."
1,[기분좋은],148495,100번 들어도 기분좋은 가요,"[208186, 155952, 669617, 438439, 397412, 60398...",4,2016-10-31 13:59:41.000,14,"[0.0573148839, -0.010279955300000001, 0.900520..."
2,"[겨울, 크리스마스]",117162,12월의 재즈,"[407630, 235568, 352044, 294512, 309377, 17965...",353,2016-12-26 12:33:33.000,8,"[-0.6645728946, -0.7428655624, 0.9763435721, -..."
3,[발라드],151638,12월의 크리스마스를 yeah,"[290351, 404382, 18938, 676958, 374448, 62388,...",29,2009-12-22 09:11:45.000,29,"[-0.0026101221, 0.1385624856, 1.9987051487, 0...."
4,[일렉],27984,1월의 최신 EDM 소식,"[34851, 596152, 650671, 16261]",84,2015-01-29 16:22:19.000,4,"[-0.4456211329, 0.0304336753, 1.2807025909, -0..."
5,"[감성새벽, 보고싶다]",29795,2000년대 발라드 모음,"[316497, 369686, 352313, 222763, 98271, 193354...",2,2020-03-16 07:36:25.000,49,"[0.0343684629, -0.4145854712, 0.7592766881, 0...."
6,[발라드],87800,2003년에 나온 다시들어도 좋은곡들,"[409746, 391858, 254435, 593128, 307044, 41151...",5,2006-06-26 14:48:06.000,66,"[-0.06574101, -0.1160902679, 0.791459024, 0.18..."
7,"[한국대중음악상, 뮤지션]",99366,2004 2019년 한국대중음악상 올해의 음악인 역대수상자,"[150714, 670156, 246743, 1236, 23520, 325909, ...",6,2020-02-07 13:15:09.000,19,"[-0.7382004857000001, -0.17712807660000002, 1...."
8,[락],2397,2008 그랜드민트 페스티벌,"[564377, 574424, 86502, 264929, 464767, 472798...",8,2008-12-24 13:41:38.000,25,"[-0.1292974651, 0.07028008250000001, 1.0626025..."
9,[댄스],25687,2009 1st songs,"[425758, 371829, 148992, 387621, 407044, 13049...",4,2009-12-25 21:55:13.000,17,"[-0.9168706536000001, -0.1231855676, 0.9921250..."


> song meta data

In [23]:
sm = pd.read_json('../db_song_meta.json')
sm.head(10)

,id,song_name,artist_name_lst,song_gnr_lsts
0,0,Feelings,[Various Artists],[POP]
1,1,"Bach : Partita No. 4 In D Major, BWV 828 - II....",[Murray Perahia],[클래식]
2,2,Solsbury Hill (Remastered 2002),[Peter Gabriel],[POP]
3,3,Feeling Right (Everything Is Nice) (Feat. Popc...,[Matoma],[일렉트로니카]
4,4,그남자 그여자,[Jude Law],[뉴에이지]
5,5,Para Los Enamorados,"[Bye, Lupita]",[재즈]
6,6,Sibelius : Valse Triste Op.44 (시벨리우스 : 슬픈 왈츠 작...,"[Herbert Blomstedt, San Francisco Symphony]",[클래식]
7,7,Superman March (From &#34;Superman&#34; / Live...,"[Los Angeles Philharmonic Orchestra, Gustavo D...",[클래식]
8,8,Lovers’ Leap (Feat. Qypthone),[애플스],[랩/힙합]
9,9,"사랑, 그대라는 멜로디",[진호],[발라드]


> gerne data

In [8]:
gnr = pd.read_json('../../kakao-arena-melon-master/melon-data/genre_gn_all.json', typ='dictionary')
gnr = pd.DataFrame(gnr, columns = ['gnr_name']).reset_index().rename(columns = {'index' : 'gnr_code'})
gnr = gnr[gnr['gnr_code'].str[-2:]=='00'].reset_index(drop=True)
gnr

,gnr_code,gnr_name
0,GN0100,발라드
1,GN0200,댄스
2,GN0300,랩/힙합
3,GN0400,R&B/Soul
4,GN0500,인디음악
5,GN0600,록/메탈
6,GN0700,성인가요
7,GN0800,포크/블루스
8,GN0900,POP
9,GN1000,록/메탈


In [9]:
gnr_mapping = dict(zip(gnr['gnr_code'], gnr['gnr_name']))
gnr_mapping

{'GN0100': '발라드',
 'GN0200': '댄스',
 'GN0300': '랩/힙합',
 'GN0400': 'R&B/Soul',
 'GN0500': '인디음악',
 'GN0600': '록/메탈',
 'GN0700': '성인가요',
 'GN0800': '포크/블루스',
 'GN0900': 'POP',
 'GN1000': '록/메탈',
 'GN1100': '일렉트로니카',
 'GN1200': '랩/힙합',
 'GN1300': 'R&B/Soul',
 'GN1400': '포크/블루스/컨트리',
 'GN1500': 'OST',
 'GN1600': '클래식',
 'GN1700': '재즈',
 'GN1800': '뉴에이지',
 'GN1900': 'J-POP',
 'GN2000': '월드뮤직',
 'GN2100': 'CCM',
 'GN2200': '어린이/태교',
 'GN2300': '종교음악',
 'GN2400': '국악',
 'GN2500': '아이돌',
 'GN2600': '일렉트로니카',
 'GN2700': 'EDM',
 'GN2800': '뮤직테라피',
 'GN2900': '뮤지컬',
 'GN3000': '크리스마스'}

### 테이블 별 데이터 프레임 전처리

In [10]:
# ply_meta 테이블:
#     ply_id int not null primary key,
#     ply_title varchar(500),
#     ply_tag_lst TEXT,
#     ply_like_cnt int,
#     ply_updt varchar(100)

ply_meta = df[['id', 'plylst_title', 'tags', 'like_cnt', 'updt_date']].rename(columns={'id':'ply_id', 'plylst_title':'ply_title', 'tags':'ply_tag_lst', 'like_cnt':'ply_like_cnt', 'updt_date':'ply_updt'})
ply_meta.head()

,ply_id,ply_title,ply_tag_lst,ply_like_cnt,ply_updt
0,50564,1 HITS OF DECADE SPECIAL 2,[팝],5,2009-07-11 21:37:42.000
1,148495,100번 들어도 기분좋은 가요,[기분좋은],4,2016-10-31 13:59:41.000
2,117162,12월의 재즈,"[겨울, 크리스마스]",353,2016-12-26 12:33:33.000
3,151638,12월의 크리스마스를 yeah,[발라드],29,2009-12-22 09:11:45.000
4,27984,1월의 최신 EDM 소식,[일렉],84,2015-01-29 16:22:19.000


In [11]:
# 태그 리스트 데이터 타입 변환
ply_meta['ply_tag_lst'] = ply_meta['ply_tag_lst'].apply(lambda x: ', '.join(x))
ply_meta.head(10)

,ply_id,ply_title,ply_tag_lst,ply_like_cnt,ply_updt
0,50564,1 HITS OF DECADE SPECIAL 2,팝,5,2009-07-11 21:37:42.000
1,148495,100번 들어도 기분좋은 가요,기분좋은,4,2016-10-31 13:59:41.000
2,117162,12월의 재즈,"겨울, 크리스마스",353,2016-12-26 12:33:33.000
3,151638,12월의 크리스마스를 yeah,발라드,29,2009-12-22 09:11:45.000
4,27984,1월의 최신 EDM 소식,일렉,84,2015-01-29 16:22:19.000
5,29795,2000년대 발라드 모음,"감성새벽, 보고싶다",2,2020-03-16 07:36:25.000
6,87800,2003년에 나온 다시들어도 좋은곡들,발라드,5,2006-06-26 14:48:06.000
7,99366,2004 2019년 한국대중음악상 올해의 음악인 역대수상자,"한국대중음악상, 뮤지션",6,2020-02-07 13:15:09.000
8,2397,2008 그랜드민트 페스티벌,락,8,2008-12-24 13:41:38.000
9,25687,2009 1st songs,댄스,4,2009-12-25 21:55:13.000


In [14]:
# ply_title_embedding 테이블:
# 	  emd_id int not null primary key auto_increment,
# 	  ply_id int not null,
#     ply_title_emd TEXT,
#     FOREIGN KEY(ply_id) REFERENCES ply_meta(ply_id)

ply_title_embedding = df[['id', 'embedding']].rename(columns={'id':'ply_id', 'embedding':'ply_title_emd'}).sort_values('ply_id').reset_index(drop=True)
ply_title_embedding.insert(0, 'emd_id', list(range(len(ply_title_embedding))))
ply_title_embedding

,emd_id,ply_id,ply_title_emd
0,0,1,"[0.2674474418, 0.031227225400000003, 1.1137478..."
1,1,2,"[-0.2025989741, -0.1607301384, 1.1295073032, 0..."
2,2,4,"[-0.3712018132, -0.1008524075, 0.8518314958000..."
3,3,5,"[-0.1932400614, -0.035883866300000004, 1.09548..."
4,4,6,"[0.2092925012, -0.21201041340000001, 0.6818352..."
...,...,...,...
76304,76304,153419,"[-0.332388401, -0.0326045789, 0.7040730119, -0..."
76305,76305,153423,"[0.0849867165, -0.10379723460000001, 1.3090186..."
76306,76306,153426,"[-0.0526723042, -0.0025272628, 0.8692131639, -..."
76307,76307,153427,"[0.1879945695, -0.0167148858, 0.642965138, 0.1..."


In [15]:
# 주의: 문자형으로 변경하여 넣은 것이기에 이후 DB에서 가져올 때 다시 float64형으로 변환 필요
ply_title_embedding['ply_title_emd'] = ply_title_embedding['ply_title_emd'].apply(lambda x: ', '.join(map(str, x)))
ply_title_embedding

,emd_id,ply_id,ply_title_emd
0,0,1,"0.2674474418, 0.031227225400000003, 1.11374783..."
1,1,2,"-0.2025989741, -0.1607301384, 1.1295073032, 0...."
2,2,4,"-0.3712018132, -0.1008524075, 0.85183149580000..."
3,3,5,"-0.1932400614, -0.035883866300000004, 1.095482..."
4,4,6,"0.2092925012, -0.21201041340000001, 0.68183529..."
...,...,...,...
76304,76304,153419,"-0.332388401, -0.0326045789, 0.7040730119, -0...."
76305,76305,153423,"0.0849867165, -0.10379723460000001, 1.30901861..."
76306,76306,153426,"-0.0526723042, -0.0025272628, 0.8692131639, -0..."
76307,76307,153427,"0.1879945695, -0.0167148858, 0.642965138, 0.17..."


In [17]:
ply_title_embedding.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 76309 entries, 0 to 76308
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   emd_id         76309 non-null  int64 
 1   ply_id         76309 non-null  int64 
 2   ply_title_emd  76309 non-null  object
dtypes: int64(2), object(1)
memory usage: 1.7+ MB


In [16]:
ply_title_embedding.nunique() # 임베딩에 중복값이 있다...? 중복되는 타이틀...? 

emd_id           76309
ply_id           76309
ply_title_emd    76247
dtype: int64

In [27]:
# song_meta 테이블:
# 	  song_id int not null primary key,
#     song_name varchar(500),
#     artist_name_lst text,
#     song_gnr_lst text

song_meta = sm
song_meta.rename(columns={'id':'song_id', 'song_gnr_lsts':'song_gnr_lst'}, inplace=True)
song_meta.head()

,song_id,song_name,artist_name_lst,song_gnr_lst
0,0,Feelings,[Various Artists],[POP]
1,1,"Bach : Partita No. 4 In D Major, BWV 828 - II....",[Murray Perahia],[클래식]
2,2,Solsbury Hill (Remastered 2002),[Peter Gabriel],[POP]
3,3,Feeling Right (Everything Is Nice) (Feat. Popc...,[Matoma],[일렉트로니카]
4,4,그남자 그여자,[Jude Law],[뉴에이지]


In [28]:
song_meta['artist_name_lst'] = song_meta['artist_name_lst'].apply(lambda x: ', '.join(x))
song_meta['song_gnr_lst'] = song_meta['song_gnr_lst'].apply(lambda x: ', '.join(x))
song_meta.head(30)

,song_id,song_name,artist_name_lst,song_gnr_lst
0,0,Feelings,Various Artists,POP
1,1,"Bach : Partita No. 4 In D Major, BWV 828 - II....",Murray Perahia,클래식
2,2,Solsbury Hill (Remastered 2002),Peter Gabriel,POP
3,3,Feeling Right (Everything Is Nice) (Feat. Popc...,Matoma,일렉트로니카
4,4,그남자 그여자,Jude Law,뉴에이지
5,5,Para Los Enamorados,"Bye, Lupita",재즈
6,6,Sibelius : Valse Triste Op.44 (시벨리우스 : 슬픈 왈츠 작...,"Herbert Blomstedt, San Francisco Symphony",클래식
7,7,Superman March (From &#34;Superman&#34; / Live...,"Los Angeles Philharmonic Orchestra, Gustavo Du...",클래식
8,8,Lovers’ Leap (Feat. Qypthone),애플스,랩/힙합
9,9,"사랑, 그대라는 멜로디",진호,발라드


In [31]:
import numpy as np
# create table song_in_ply 테이블:
# 	  songinply_id int not null primary key auto_increment,
# 	  ply_id int,
#     song_id int,
#     FOREIGN KEY(ply_id) REFERENCES ply_meta(ply_id),
#     FOREIGN KEY(song_id) REFERENCES song_meta(song_id)

sip = pd.DataFrame(df[['id', 'songs']])
sip
# sip_unnest = np.dstack((np.repeat(sip.id.values, list(map(len, sip.songs))), np.concatenate(sip.songs.values)))
# song_in_ply = pd.DataFrame(data = sip_unnest[0], columns = sip.columns).rename(columns={'id':'ply_id', 'songs':'song_id'})
# song_in_ply

,id,songs
0,50564,"[60025, 326802, 82991, 533287, 132109, 298235,..."
1,148495,"[208186, 155952, 669617, 438439, 397412, 60398..."
2,117162,"[407630, 235568, 352044, 294512, 309377, 17965..."
3,151638,"[290351, 404382, 18938, 676958, 374448, 62388,..."
4,27984,"[34851, 596152, 650671, 16261]"
...,...,...
76304,31642,"[7730, 356025, 172750, 555157, 34548, 470306, ..."
76305,56144,"[183424, 431998, 354592, 241327, 536220, 58465..."
76306,110536,"[349306, 93890, 651902, 46845, 362347, 353573,..."
76307,106660,"[79329, 173875, 216115, 422584, 147482, 277739..."


In [30]:
n = 0
for i in range(len(sip)):
    if sip['id'][i] not in ply_meta['ply_id']:
        n += 1
print(n)

38394


In [269]:
# train 데이터 songs에 있는 song들과 song meta 전처리한 song들이 동일한가...?

for i in range(len(song_in_ply)):
    if song_in_ply['song_id'][i] not in sm2['song_id']:
        # song_in_ply.drop([i], axis=0, inplace=True)
        song_in_ply = song_in_ply.drop([i], axis=0)

song_in_ply

,ply_id,song_id
0,50564,60025
1,50564,326802
2,50564,82991
3,50564,533287
4,50564,132109
...,...,...
1869662,150419,73417
1869663,150419,102479
1869664,150419,334620
1869665,150419,217063


In [279]:
# song_in_ply= song_in_ply.reset_index(drop=True)

n = 0
for i in range(len(song_in_ply)):
    if song_in_ply['ply_id'][i] not in ply_meta['ply_id']:
        n += 1
print(n)


932615


### MySQL 연동

In [33]:
from sqlalchemy import create_engine
import pymysql

In [ ]:
engine = create_engine("mysql+pymysql://유저:"+"비밀번호"+"@ip주소:포트번호/DB명?charset=utf8")
conn = engine.connect()

In [52]:
# ply_meta 테이블에 데이터프레임 저장
ply_meta.to_sql(name='ply_meta', con=engine, if_exists='append', index=False)

76309

In [235]:
# ply_title_embedding 테이블에 데이터프레임 저장
# ply_title_embedding.to_sql(name='ply_title_embedding', con=engine, if_exists='append', index=False)

data_to_insert = ply_title_embedding[['ply_id', 'ply_title_emd']]
data_to_insert.to_sql(name='ply_title_embedding', con=engine, if_exists='append', index=False)

76309

In [198]:
# song_meta 테이블에 데이터프레임 저장
sm2.to_sql(name='song_meta', con=engine, if_exists='append', index=False)

703945

In [271]:
# song_in_ply 테이블에 데이터프레임 저장
song_in_ply.to_sql(name='song_in_ply', con=engine, if_exists='append', index=False)

IntegrityError: (pymysql.err.IntegrityError) (1452, 'Cannot add or update a child row: a foreign key constraint fails (`test`.`song_in_ply`, CONSTRAINT `song_in_ply_ibfk_2` FOREIGN KEY (`song_id`) REFERENCES `song_meta` (`song_id`))')
[SQL: INSERT INTO song_in_ply (ply_id, song_id) VALUES (%(ply_id)s, %(song_id)s)]
[parameters: [{'ply_id': 50564, 'song_id': 60025}, {'ply_id': 50564, 'song_id': 326802}, {'ply_id': 50564, 'song_id': 82991}, {'ply_id': 50564, 'song_id': 533287}, {'ply_id': 50564, 'song_id': 132109}, {'ply_id': 50564, 'song_id': 298235}, {'ply_id': 50564, 'song_id': 260221}, {'ply_id': 50564, 'song_id': 123794}  ... displaying 10 of 1858497 total bound parameter sets ...  {'ply_id': 150419, 'song_id': 217063}, {'ply_id': 150419, 'song_id': 436029}]]
(Background on this error at: https://sqlalche.me/e/20/gkpj)

In [54]:
conn.close()

In [34]:
# mysql 연동
pms_conn = pymysql.connect(host='IP'
                , port=포트번호
                , user = '유저'
                , password='비밀번호'
                , db = 'DB명'
                , charset='utf8'
                ) 

cur = pms_conn.cursor()

In [42]:
# DB에서 데이터 가져오기
SQL = "SELECT * FROM ply_meta;"
temp = pd.read_sql(SQL, pms_conn)
temp

C:\Users\Sanai_YMR\AppData\Local\Temp\ipykernel_5552\2974487717.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  temp = pd.read_sql(SQL, pms_conn)


,ply_id,ply_title,ply_tag_lst,ply_like_cnt,ply_updt
0,1,________ 질리지 않는 이 곡 들,"힙합, 랩",2,2011-11-22 17:31:01.000
1,2,좋은 꿈을 꿀것같은 사랑스러운 음악,"밤, 새벽",11,2014-07-28 12:30:32.000
2,4,9회말 2아웃 1픽으로 장만하는 데뷔의 발판 프로듀스 101 시즌2,"tv별책부록, 프로듀스101, 프로듀스101시즌2",5,2017-06-16 17:20:40.000
3,5,3월 봄이오는 소식과 함께 겨울 안뇽,"감성, 카페, 봄노래, 드라이브, 여행, 휴식, 기분전환, 사랑, 힐링, 신나는",6,2020-03-10 18:11:12.000
4,6,조깅할때나 운동할때 신나는 EDM,"헬스음악, 헬스장, 운동할때신나는음악, 자전거, 운동할때듣는음악, 라이딩, 운동할때",180,2020-04-18 12:28:33.000
...,...,...,...,...,...
76304,153419,생생한 고음질로 만나는 Hi Fi 가요 위클리 11월 3일자,"신곡, 가요, 믿고듣는, 드라이브, 기분전환, 최신, 트렌디",6,2017-11-03 18:03:24.000
76305,153423,당신의 봄은 언제인가요 감성폭발 봄맞이송 20선,"취향저격DJ, 봄, 봄바람",22,2019-03-06 12:09:34.000
76306,153426,인범 랩 흑인랩 클럽 댄스 옛클럽 클럽댄스,"힙합, 랩",22,2010-06-16 15:08:13.000
76307,153427,리드미컬한 락 모음,락,2,2018-09-30 10:31:14.000


In [43]:
# 문자형으로 DB에 저장했던 ply_tag_lst 컬럼을 다시 리스트화
temp['ply_tag_lst'] = temp['ply_tag_lst'].apply(lambda x: x.split(','))
temp.head(10)

,ply_id,ply_title,ply_tag_lst,ply_like_cnt,ply_updt
0,1,________ 질리지 않는 이 곡 들,"[힙합, 랩]",2,2011-11-22 17:31:01.000
1,2,좋은 꿈을 꿀것같은 사랑스러운 음악,"[밤, 새벽]",11,2014-07-28 12:30:32.000
2,4,9회말 2아웃 1픽으로 장만하는 데뷔의 발판 프로듀스 101 시즌2,"[tv별책부록, 프로듀스101, 프로듀스101시즌2]",5,2017-06-16 17:20:40.000
3,5,3월 봄이오는 소식과 함께 겨울 안뇽,"[감성, 카페, 봄노래, 드라이브, 여행, 휴식, 기분전환, 사랑, ...",6,2020-03-10 18:11:12.000
4,6,조깅할때나 운동할때 신나는 EDM,"[헬스음악, 헬스장, 운동할때신나는음악, 자전거, 운동할때듣는음악, 라이딩...",180,2020-04-18 12:28:33.000
5,8,안되는걸 알면서도 희망을 갖고사는나,"[비오는날, 슬픔, 이별]",13,2015-09-06 22:33:37.000
6,9,괜히 울적할 때 들으면 좋은 노래,"[감성, 드라이브, 가을]",5,2018-10-06 20:58:20.000
7,12,울림엔터테인먼트의 울림있는 노래,"[기분전환, 까페]",6,2016-01-21 11:42:44.000
8,13,따뜻한 홍차라떼와 어울리는 음악들,"[휴식, 힐링, 설렘, 사랑]",5,2016-01-22 11:13:03.000
9,14,오우 POP 좀 들을 줄 아는 놈인가,[팝송],12,2019-09-16 16:22:58.000


In [47]:
# 확인
temp['ply_tag_lst'][0]
temp['ply_tag_lst'][0][1]

' 랩'

In [60]:
pms_conn.close()